In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-15 23:56:07--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-15 23:56:07 (33.1 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [3]:
import minsearch

In [4]:
# import requests 

# docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
# docs_response = requests.get(docs_url)
# documents_raw = docs_response.json()

# documents = []

# for course in documents_raw:
#     course_name = course['course']

#     for doc in course['documents']:
#         doc['course'] = course_name
#         documents.append(doc)

# import json
# with open('documents.json', 'w', encoding='utf-8') as json_file:
#     json.dump(documents, json_file, ensure_ascii=False, indent=4)

In [8]:
import json
with open('documents.json', 'rt', encoding='utf-8') as f_in:
    documents = json.load(f_in)

In [13]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
from minsearch import Index

index = Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [13]:
query = 'if the course is already started, can I be enrolled?'

boost = {'question':3., 'section':.5}

results = index.search(
    query = query,
    filter_dict = {'course': 'data-engineering-zoomcamp'},
    boost_dict = boost,
    num_results = 5
)

In [14]:
results

[{'text': 'Yes, the slack channel remains open and you can ask questions there. But always sDocker containers exit code w search the channel first and second, check the FAQ (this document), most likely all your questions are already answered here.\nYou can also tag the bot @ZoomcampQABot to help you conduct the search, but don’t rely on its answers 100%, it is pretty good though.',
  'section': 'General course-related questions',
  'question': 'Course - Can I get support if I take the course in the self-paced mode?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data

In [15]:
from openai import OpenAI

In [19]:
client = OpenAI()

In [20]:
def search(query):
    boost = {'question':3., 'section':.5}

    results = index.search(
        query = query,
        filter_dict = {'course': 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 5
    )

    return results

In [21]:
def build_prompt(query, search_rusults):
    prompt_template = """
You're a course teacher assistant. Answer the QUESTION based on the CONTEXT from FAQ database. 
Use only the facts of CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
"""

    context = ''
    for doc in search_rusults:
        context = context + f"section: {doc['section']}\nquestion:{doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [22]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-3.5-turbo-0125',
        messages=[{"role":"user", "content":prompt}]
    )
    
    return response.choices[0].message.content

In [23]:
def rag(query):
    results = search(query)
    prompt = build_prompt(query, results)
    response = llm(prompt)
    return response

In [24]:
query = 'how do I run kafka?'

response = rag(query)

In [25]:
print(response)

To run Kafka, you need to follow specific steps based on the language and tools you are using. For Java, you can run the Java Kafka producer, consumer, or kstreams in the project directory by using the command: java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java. For Python, if you encounter the error "Module 'kafka' not found," you can create a virtual environment, install necessary packages from requirements.txt, and run your Python files within that environment. Additionally, for any issues related to missing modules like 'kafka.vendor.six.moves,' you can use the kafka-python-ng library instead of the original kafka-python package to resolve the error.


# ElasticSearch

In [1]:
from elasticsearch import Elasticsearch

In [2]:
es_client = Elasticsearch('http://localhost:9200')

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [6]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document = doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:20<00:00, 47.20it/s]


In [27]:
query = 'I just discover the course. Can I still join it?'

In [36]:
def elastic_searc(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [38]:
elastic_searc(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [34]:
def rag(query):
    results = elastic_searc(query)
    prompt = build_prompt(query, results)
    response = llm(prompt)
    return response

In [39]:
rag(query)

'Based on the context provided, yes, you can join the course even after the start date. You are still eligible to submit homework, but be mindful of deadlines for final projects. Additionally, you can follow the course at your own pace after it finishes, access materials, and even start working on the final capstone project.'

# Homework

In [49]:
query = "How do I execute a command in a running docker container?"
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                # }
            }
        }
    }
response = es_client.search(index=index_name, body=search_query)

In [53]:
response['hits']['max_score']

84.050095

In [72]:
query = "How do I execute a command in a running docker container?"
search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
response = es_client.search(index=index_name, body=search_query)

In [73]:
for hit in response['hits']['hits']:
    print(hit['_source']['question'])

How do I debug a docker container?
How do I copy files from my local machine to docker container?
How do I copy files from a different folder into docker container’s working directory?


In [74]:
result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [75]:
result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [76]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

context = ''
for doc in result_docs:
    context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"

prompt = prompt_template.format(question=query, context=context).strip()

In [77]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [78]:
len(prompt)

1462

In [79]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.6 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 21.4 MB/s eta 0:00:00m eta 0:00:01


In [81]:
import tiktoken

In [82]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [85]:
len(encoding.encode(prompt))

322

In [87]:
encoded_prompt = encoding.encode(prompt)

In [94]:
for code in encoded_prompt:
    print(str(encoding.decode_single_token_bytes(code)).replace('b', ''), end='')

"You're"' a'' course'' teaching'' assistant''.'' Answer'' the'' QUESTION'' ased'' on'' the'' CONT''EXT'' from'' the'' FAQ'' dataase''.\n''Use'' only'' the'' facts'' from'' the'' CONT''EXT'' when'' answering'' the'' QUESTION''.\n\n''QUESTION'':'' How'' do'' I'' execute'' a'' command'' in'' a'' running'' docker'' container''?\n\n''CON''TEXT'':\n''Q'':'' How'' do'' I'' deug'' a'' docker'' container''?\n''A'':'' Launch'' the'' container'' image'' in'' interactive'' mode'' and'' overriding'' the'' entry''point'','' so'' that'' it'' starts'' a'' ash'' command''.\n''docker'' run'' -''it'' --''entry''point'' ash'' <''image''>\n''If'' the'' container'' is'' already'' running'','' execute'' a'' command'' in'' the'' specific'' container'':\n''docker'' ps'' (''find'' the'' container''-id'')\n''docker'' exec'' -''it'' <''container''-id''>'' ash''\n''(M''ar''cos'' M''JD'')\n\n''Q'':'' How'' do'' I'' copy'' files'' from'' my'' local'' machine'' to'' docker'' container''?\n''A'':'' You'' can'' copy'' 